In [1]:

import ee
import collections
import os
import geemap
import subprocess
import math
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:33210'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:33210'
collections.Callable = collections.abc.Callable
ee.Initialize()

In [2]:
#加载本地矢量数据
HHMC4_DZG = 'D:/hhmc_roi/hhmc2/HHMC_SD/HHMC_SD_p1.shp'
HHMC4_DZG = geemap.shp_to_ee(HHMC4_DZG)
HHMC4_DZG = HHMC4_DZG.geometry()
Map = geemap.Map()
Map.addLayer(HHMC4_DZG)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
tide_roi1 = Map.draw_last_feature
tide_roi1 = tide_roi1.geometry()

In [6]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")#.map(rmCloudByQA)
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
#L8  = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# bandIn_S2 = ['B3','B4','B8','B11','cloud']
# bandOut = ['green','red','nir','swir1','cloud']
bandIn_S2 = ['B3','B4','B8','B2','cloud']
bandOut = ['green','red','nir','blue','cloud']
#bandIn_L8 = ['SR_B3','SR_B4','SR_B5','SR_B6','QA_PIXEL']
bandIn_L8 = ['B3','B4','B5','B6','pixel_qa']

In [7]:
def get_s2_sr_cld_col(aoi, start_date, end_date,MGRS_TILE,SENSING_ORBIT_NUMBER,CLOUDY_PIXEL_PERCENTAGE):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterBounds(aoi)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.eq('MGRS_TILE',MGRS_TILE))\
        .filter(ee.Filter.eq('SENSING_ORBIT_NUMBER',SENSING_ORBIT_NUMBER))\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUDY_PIXEL_PERCENTAGE)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')\
        .filterBounds(aoi)\
        .filterDate(start_date, end_date))\
        

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.lte(60).rename('cloud')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def cloud_mask_L8(img):
    qa = img.select('cloud')
    cloudShadowBitMask = 1<<3
    cloudsBitMask = 1<<5
    cloud = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    cloud = cloud.rename('cloud')
    return img.addBands(cloud,None,True)

def addtime(img):
    date = ee.Date(img.date())
    doy = ee.Number.parse(date.format('D'))
    img = img.set('DOY',doy)
    return img.addBands(ee.Image(doy).rename('doy'),None,True).toFloat()

def CLIP(img):
    return img.clip(tide_roi1).unmask(1)

def applyScaleFactors(img):
    opticalBands = img.select('SR_B.').multiply(0.275).add(-2000).toInt()
    return img.addBands(opticalBands,None,True)
    return img

In [31]:
start_date = '2021-01-01'
end_date = '2021-12-17'
MGRS_TILE = '50SPG'
SENSING_ORBIT_NUMBER = 132
CLOUDY_PIXEL_PERCENTAGE = 30
WRS_PATH = 121
WRS_ROW = 34

s2_sr_cld_col_eval = get_s2_sr_cld_col(tide_roi1, start_date, end_date,MGRS_TILE,SENSING_ORBIT_NUMBER,CLOUDY_PIXEL_PERCENTAGE)
s2_sr_cld_col_eval = s2_sr_cld_col_eval.map(add_cloud_bands)
S2_datesat_2021 = s2_sr_cld_col_eval.select(bandIn_S2,bandOut)

S2_datesat_2021 = S2_datesat_2021.filterBounds(tide_roi1)\
            .filterDate(start_date,end_date)\
            .filter(ee.Filter.eq('MGRS_TILE',MGRS_TILE))\
            .filter(ee.Filter.eq('SENSING_ORBIT_NUMBER',SENSING_ORBIT_NUMBER))\
            .map(addtime)\
            .map(CLIP).filter(ee.Filter.neq('DOY',326))

L8_datesat_2021 = L8.filterBounds(tide_roi1)\
            .filterDate(start_date,end_date)\
            .filter(ee.Filter.lt('CLOUD_COVER',CLOUDY_PIXEL_PERCENTAGE))\
            .filter(ee.Filter.eq('WRS_PATH',WRS_PATH))\
            .filter(ee.Filter.eq('WRS_ROW',WRS_ROW))\
            .select(bandIn_L8,bandOut)\
            .map(cloud_mask_L8)\
            .map(addtime)\
            .map(CLIP)\
            .filter(ee.Filter.neq('DOY',318)).filter(ee.Filter.neq('DOY',316)).filter(ee.Filter.neq('DOY',348))
            
viz = {'bands':['nir','red','green'],'min':0,'max':4000}


In [32]:
l8_list = L8_datesat_2021.toList(L8_datesat_2021.size())
# refer_img = ee.Image(l8_list.get(0))
# refer_img_s2 = S2_datesat_2021.first()
S2_list = S2_datesat_2021.toList(L8_datesat_2021.size())
refer_img = ee.Image(S2_list.get(2))


def Spatial_register(img):
    
    referRedBand = refer_img.select('red')
    imageRedBand = img.select('red')
    
    displacement = imageRedBand.displacement(
      referenceImage = referRedBand,
      maxOffset = 50.0,
      patchWidth = 100.0
    )

    registered = img.displace(displacement).copyProperties(img)
    registered = ee.Image(registered.copyProperties(img))
    
    return registered
    
def resample_L8(img):

    crs = refer_img.select('red').projection().crs()
    resampled = img.resample('bicubic').reproject(crs = crs,scale = 10) 
    resampled = addtime(resampled)
    
    return resampled.copyProperties(img)


SlopeL8 = [1.0296,1.0661,0.9804,1.0326,1,1]
offsetL8 = [17,-13,39,101,0,0]

def band_adjust(img):
    imgL8SR_bandadj = img.multiply(SlopeL8).add(offsetL8).multiply(1000).toInt().divide(1000)
    return imgL8SR_bandadj
    
def addNDWI(img):
    ndwi = img.normalizedDifference(['green','nir']).rename('NDWI')
    return img.addBands(ndwi)

def addNDVI(img): 
    ndvi = img.normalizedDifference(['nir','red']).rename('NDVI')   
    return img.addBands(ndvi)


In [33]:
L8_LIST = L8_datesat_2021.toList(L8_datesat_2021.size())
img = ee.Image(L8_LIST.get(1))
registered = Spatial_register(img)
resampled = ee.Image(resample_L8(registered))
   # registered = Spatial_register(resampled) 



Map.addLayer(img,viz,'img')
Map.addLayer(resampled,viz,'resampled')

In [17]:
Map

Map(bottom=3240974.0, center=[37.83249699754951, 119.23543453216554], controls=(WidgetControl(options=['positi…

In [34]:
mlist = ee.List([])
S2_LIST = S2_datesat_2021.toList(S2_datesat_2021.size())
n1 = S2_datesat_2021.size().getInfo()
L8_LIST = L8_datesat_2021.toList(L8_datesat_2021.size())
n2 = L8_datesat_2021.size().getInfo()
#添加S2
mlist = ee.List([])
for i in range(n1):
    
    img  = ee.Image(S2_LIST.get(i))
    #resampled = ee.Image(resample_S2(img))
    #registered = Spatial_register(img)
    #img = ee.Image(registered).multiply(1000).toInt().divide(1000)#.unmask(1)
    mlist = mlist.add(img)
    
for i in range(n2):
    
    img = ee.Image(L8_LIST.get(i))
    #resampled = ee.Image(resample_L8(img))
    registered = Spatial_register(img) 
    resampled = addtime(registered)
    harmoic_L8 = band_adjust(resampled).multiply(1000).toInt().divide(1000)#.unmask(1)
    mlist = mlist.add(harmoic_L8)
    
collect = ee.ImageCollection.fromImages(mlist)


doy_array = collect.toArray().arraySlice(1,5,6)
sorted_array =  collect.toArray().arraySort(doy_array)

n = collect.size().getInfo()
name = [str(j) for j in range(n)]

#M = geemap.Map()

# bandname = ['green','red','nir','swir1','cloud','doy']
bandname = ['green','red','nir','blue','cloud','doy']
img_list = ee.List([])
for i in range(n):
    img = sorted_array.arraySlice(0,i,i+1).arrayProject([1]).arrayFlatten([bandname])
    #M.addLayer(img,viz,'img'+str(i))
    img_list = img_list.add(img)
    
datesat_2021 = ee.ImageCollection.fromImages(img_list)

#collect = collect#.map(addNDVI).map(addNDWI)


In [35]:
def bc(img):
    return img.unmask(0).clip(tide_roi1)

raw_collection = datesat_2021.map(bc)#.map(addNDWI)

# 去云区域线性插值
def linear_interpolate(img,img_before,img_after):
    #生成去云掩膜
    mask = (img.select('cloud')).lte(0)
    unmask = (img.select('cloud')).gt(0)
    #线性插值
    doy_b = img_before.select('doy')
    doy_a = img_after.select('doy')
    doy = img.select('doy')
    
    k = (img_after.subtract(img_before)).divide(doy_a.subtract(doy_b))
    x = doy.subtract(doy_b)
    ch = (x.multiply(k))
    
    linear = (img_before.add(ch))#.select(['B8','B5','B4','B3','B2','NDVI','NDWI','EVI','doy'])
    linear = linear.updateMask(mask)
    remain = (img.updateMask(unmask))#.select(['B8','B5','B4','B3','B2','NDVI','NDWI','EVI','doy'])
    
    #数据融合
    f = ee.ImageCollection([linear,remain])#image
    
    return f.mosaic().multiply(1000).toInt().divide(1000) 

#对2021数据进行去云插值
img_list_2021 = datesat_2021.toList(datesat_2021.size())

#datesat_QY represent datesat after remove cloud (clip by roi)
n = datesat_2021.size().getInfo()
datesat_QY_2021 = ee.ImageCollection(ee.Image(img_list_2021.get(0)).multiply(1000).toInt().divide(1000))

for k in range(1,n-1):
    img = ee.Image(img_list_2021.get(k))
    img1 = ee.Image(img_list_2021.get(k-1))
    img2 = ee.Image(img_list_2021.get(k+1))
    image = ee.ImageCollection(linear_interpolate(img,img1,img2))
    datesat_QY_2021 = datesat_QY_2021.merge(image)

datesat_QY_2021 = datesat_QY_2021.merge(ee.ImageCollection(ee.Image(img_list_2021.get(n-1)).multiply(1000).toInt().divide(1000)))
datesat_QY_2021 = datesat_QY_2021.map(addNDVI).map(addNDWI)
raw_datesat = datesat_QY_2021
datesat_QY_2021 = datesat_QY_2021.select(['doy','NDVI','NDWI'])


datesat_QY_2021 = datesat_QY_2021.map(bc)

In [36]:
#name_No = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']
# n = datesat_QY.size().getInfo()
# name_No = [k for k in range(0,n)]
# name_No = [str(k) for k in name_No]

def get_weight(datesat,i_obj,tide_flag,k):
    
    bands = ['doy','NDVI','NDWI']
    
    i = i_obj # i represent No of reconstructed Image
    n = k  # k represent sum of reconstruct Image
    # tide_flag  :if tide_flag == 1,mean neap tide region ,else mean spring tide region
    
    num = datesat.size().getInfo()
    No = [j for j in range(0,num)]
    name_No = [str(j) for j in No]

    datesat = datesat.select(bands)
    datesat_list = datesat.toList(datesat.size())
    
    datesat_array = datesat.toArray()
    datesat_doy = (datesat_array.arraySlice(1,0,1).arrayProject([0])).arrayFlatten([name_No])
    datesat_NDVI = (datesat_array.arraySlice(1,1,2).arrayProject([0])).arrayFlatten([name_No])
    T = (datesat_doy.select(name_No[i+n])).subtract(datesat_doy.select(name_No[i-n]))
    t0 = datesat_doy.select(name_No[i])
    
    ############### create quality information for reconstruct Images by using NDWI index      ###############
    #u represent 参数
    u = 1
    #将NDWI大于0的值变为0，小于0的值改为1
    datesat_NDWI =  (datesat_array.arraySlice(1,2,3).arrayProject([0])).arrayFlatten([name_No])
    #datesat_NDWI = (ee.Image.constant(1)).divide(datesat_NDWI.multiply(u).exp().add(1))
    datesat_NDWI_weight = datesat_NDWI.where(datesat_NDWI.gt(0),0)
    datesat_NDWI_weight = datesat_NDWI.where(datesat_NDWI.lt(0),1)
    
    #####################################
                                                                  
    #  自动生成：weit_name = ['w1','w2','w3','w4','w5','w6']
    weit_name = [k for k in range(1,2*n+1)]
    weit_name = ['w'+str(k) for k in weit_name]
                                                                  
    weit_C = ee.Image.constant(ee.List.repeat(1,2*n)).float().rename(weit_name)
    
    if tide_flag == 1:
        for j in range(1,n+1):
            # 时间变量 t = (t - t0)/(T - S)
            t_b = ((datesat_doy.select(name_No[i-j]).subtract(t0)).divide(T)).float()
            t_a = ((datesat_doy.select(name_No[i+j]).subtract(t0)).divide(T)).float()
            # NDWI 归一化变量（0 /1）
            Qndwi_b = (datesat_NDWI_weight.select(name_No[i-j]))
            Qndwi_a = (datesat_NDWI_weight.select(name_No[i+j]))
            
            ndwi_b = (datesat_NDWI.select(name_No[i-j]))
            ndwi_a = (datesat_NDWI.select(name_No[i+j]))
            
            wa = weit_C.select('w'+str(j+n)).multiply(t_a)
            wa = (((wa.multiply(ndwi_a).divide(wa.abs())).subtract((wa.pow(2)).divide(0.2))).exp()).multiply(1000).toInt().divide(1000)
            wa = wa.multiply(Qndwi_a)
            
            wb = weit_C.select('w'+str(n+1-j)).multiply(t_b)
            wb = (((wb.multiply(ndwi_b).divide(wb.abs())).subtract((wb.pow(2)).divide(0.2))).exp()).multiply(1000).toInt().divide(1000)
            wb = wa.multiply(Qndwi_b)
            
            wb = wb.rename('w'+str(n+1-j))
            wa = wa.rename('w'+str(j+n))
            weit_C = weit_C.addBands(wa,None,True)\
                           .addBands(wb,None,True)     
    else:
        for j in range(1,n+1):
            
            t_b = ((datesat_doy.select(name_No[i-j]).subtract(t0)).divide(T)).float()
            t_a = ((datesat_doy.select(name_No[i+j]).subtract(t0)).divide(T)).float()
            # NDWI 归一化变量（0 /1）
            Qndwi_b = (datesat_NDWI_weight.select(name_No[i-j]))
            Qndwi_a = (datesat_NDWI_weight.select(name_No[i+j]))
            
            wa = weit_C.select('w'+str(j+n)).multiply(t_a)
            wa = (((wa.pow(2)).divide(0.2).multiply(-1)).exp()).multiply(1000).toInt().divide(1000)  
            wa = wa.multiply(Qndwi_a)
            
            wb = weit_C.select('w'+str(n+1-j)).multiply(t_b)
            wb = (((wb.pow(2)).divide(0.2).multiply(-1)).exp()).multiply(1000).toInt().divide(1000)
            wb = wa.multiply(Qndwi_b)
            
            wb = wb.rename('w'+str(n+1-j))
            wa = wa.rename('w'+str(j+n))
            weit_C = weit_C.addBands(wa,None,True)\
                           .addBands(wb,None,True)
            
    def weit_Normalization(weit_img,n):
        
        sum_weit = (weit_img.toArray().toArray(1)).arrayAccum(0).arrayGet([2*n-1,0])
        Weight = (weit_img.divide(sum_weit)).multiply(1000).toInt().divide(1000) 
    
        return Weight
    
    Weit_img = weit_Normalization(weit_C,n)    
                      
    return Weit_img


def tide_gap_Filling(datesat,No,Bands,k):
    
    # datesat ---> input datesat
    # weight_img ---> weight image 
    # No ---> number of reconstructed Image in datesat                                                              
    # Bands ---> the bands you want gap filling
    # k ---> number of reconstruct Image for gap filling (default = 3)
    
    i = No
        
    No_B = [y for y in range(1,k+1)]
    No_B = [str(y) for y in No_B] #    No_B = ['1','2','3']
    No_A = [y for y in range(k+1,2*k+1)]
    No_A = [str(y) for y in No_A] #    No_A = ['4','5','6']

    
    NDVI_datesat = (datesat.select(Bands)).toArray().toArray(1)
    NDVI_Before = NDVI_datesat.arraySlice(0,i-k,i).arrayProject([0]).arrayFlatten([No_B])
    NDVI_After = NDVI_datesat.arraySlice(0,i+1,i+k+1).arrayProject([0]).arrayFlatten([No_A])
    NDVI_Collection = NDVI_Before.addBands(NDVI_After)
    
    datesat_list = datesat.toList(datesat.size())
    target_img = ee.Image(datesat_list.get(No))
    tide_region = target_img.updateMask(target_img.select('NDWI').gt(0))
   
    
    #获取三种情况 非淹没区 大潮区 小潮区
    non_tide_region = target_img.updateMask(target_img.select('NDWI').lte(0))
    
    #小潮
    #Ntide_region = datesat.updateMask(tide_region.select('NDWI').lte(0.4))
    tide_thred =  0.4
    NDVI_Collection_Ntide = NDVI_Collection.updateMask(tide_region.select('NDWI').lte(tide_thred))
    #weight_img_Ntide = (get_weight(datesat,No,1,k)).updateMask(tide_region.select('NDWI').lte(tide_thred))
    weight_img_Ntide = (get_weight(datesat,No,0,k)).updateMask(tide_region.select('NDWI').lte(tide_thred))
    weighted_NDVI_Collection_Ntide = NDVI_Collection_Ntide.multiply(weight_img_Ntide).multiply(1000).toInt().divide(1000)
    composite_NDVI_Ntide = (weighted_NDVI_Collection_Ntide.toArray().toArray(1)).arrayAccum(0).arrayGet([2*k-1,0]).rename(Bands+str(i))  
    
    #大潮
   # Stide_region = datesat.updateMask(tide_region.select('NDWI').gt(0.4))
    NDVI_Collection_Stide = NDVI_Collection.updateMask(tide_region.select('NDWI').gt(tide_thred))
    #weight_img_Stide = (get_weight(datesat,No,0,k)).updateMask(tide_region.select('NDWI').gt(tide_thred))
    weight_img_Stide = (get_weight(datesat,No,1,k)).updateMask(tide_region.select('NDWI').gt(tide_thred))
    weighted_NDVI_Collection_Stide = NDVI_Collection_Stide.multiply(weight_img_Stide).multiply(1000).toInt().divide(1000)
    composite_NDVI_Stide = (weighted_NDVI_Collection_Stide.toArray().toArray(1)).arrayAccum(0).arrayGet([2*k-1,0]).rename(Bands+str(i))
    
    #非淹没区
    non_tide_region = non_tide_region.select(Bands).multiply(1000).toInt().divide(1000).rename(Bands+str(i))
    
    merge = ee.ImageCollection([composite_NDVI_Stide,composite_NDVI_Ntide,non_tide_region])
    merge = merge.mosaic().multiply(1000).toInt().divide(1000).rename('NDVI')

    return merge


In [37]:
n = datesat_QY_2021.size().getInfo()
NDVI_no = [k for k in range(0,n)]
NDVI_no = ['NDVI'+str(k) for k in NDVI_no]
print(NDVI_no)

['NDVI0', 'NDVI1', 'NDVI2', 'NDVI3', 'NDVI4', 'NDVI5', 'NDVI6', 'NDVI7', 'NDVI8', 'NDVI9', 'NDVI10', 'NDVI11', 'NDVI12', 'NDVI13', 'NDVI14', 'NDVI15', 'NDVI16', 'NDVI17', 'NDVI18', 'NDVI19', 'NDVI20', 'NDVI21', 'NDVI22', 'NDVI23', 'NDVI24', 'NDVI25', 'NDVI26', 'NDVI27', 'NDVI28', 'NDVI29', 'NDVI30', 'NDVI31', 'NDVI32', 'NDVI33', 'NDVI34', 'NDVI35', 'NDVI36', 'NDVI37', 'NDVI38', 'NDVI39', 'NDVI40', 'NDVI41', 'NDVI42', 'NDVI43']


In [38]:
def apply_to_TimeSeries(datesat,Bands):
    
    
    # apply Tide_Gap_filling to timeseries
    List = ee.List([])
    img_list = datesat.toList(datesat.size())
    n = datesat.size().getInfo()
    
    img0 = ee.Image(img_list.get(0)).select(['doy','NDVI'])
    img0 = img0.multiply(1000).toInt().divide(1000)
    img1 = ee.Image(img_list.get(1)).select(['doy']).addBands(tide_gap_Filling(datesat,1,Bands,1).rename('NDVI'))
    img2 = ee.Image(img_list.get(2)).select(['doy']).addBands(tide_gap_Filling(datesat,2,Bands,2).rename('NDVI'))
    List = ee.List([img0,img1,img2])
    
    for i in range(3,n-3):
        img =  ee.Image(img_list.get(i)).select(['doy']).addBands(tide_gap_Filling(datesat,i,Bands,3).rename('NDVI'))
        List = List.add(img)   
    
    img22 = ee.Image(img_list.get(n-3)).select(['doy']).addBands(tide_gap_Filling(datesat,n-3,Bands,2).rename('NDVI'))
    img11 = ee.Image(img_list.get(n-2)).select(['doy']).addBands(tide_gap_Filling(datesat,n-2,Bands,1).rename('NDVI'))
    img00 = ee.Image(img_list.get(n-1)).select(['doy','NDVI'])
    List = List.add(img22).add(img11).add(img00)
    
    collection = ee.ImageCollection.fromImages(List)
    
    return collection


def count_VI_dif(datesat):
    
    # input :datesat_dif
    
    n = datesat.size().getInfo()
    img_list = datesat.toList(n)
    list_dif = ee.List([])
    
    print(n)
    def VI_dif(img_before,img_after,number):
    
    #计算差值NDVI 和 对应的日期天数\

        img_before_ndvi = img_before.select('NDVI')
        img_after_ndvi = img_after.select('NDVI')
    
        NDVI_DIF = img_before_ndvi.subtract(img_after_ndvi)
        doy_mid = ((img_before.select('doy').add(img_after.select('doy'))).divide(2)).toInt()
        doy_after = img_after.select('doy')
        doy_before = img_before.select('doy')
        doy_dif = doy_after.subtract(doy_before).toInt()
        
        return NDVI_DIF.addBands(doy_mid.rename('H_doy'))\
                        .addBands(doy_dif.rename('dif_d'))\
                        .addBands(doy_after.rename('doy_after'))\
                        .addBands(doy_before.rename('doy_before'))

    for i in range(0,n-1):
        img1 = ee.Image(img_list.get(i))
        img2 = ee.Image(img_list.get(i+1))
        ndvi = VI_dif(img1,img2,0)
        list_dif = list_dif.add(ndvi)

    dif_datesat = ee.ImageCollection.fromImages(list_dif)
    
    return dif_datesat


In [39]:
Bands = 'NDVI'
datesat_QY_TGF_2021 = apply_to_TimeSeries(datesat_QY_2021,Bands)


In [40]:
def Potential_clearing_timeseries(datesat):
    
    n = datesat.size().getInfo()
    datesat_list = datesat.toList(n)
    
    if n%3 == 0:
        num = int(n/3)
    else:
        num = int(n/3)+1

    mosaic_list = ee.List([])
    
    for i in range(0,num):
        
        if i < num-1:
            s = i*3
            e = s+3
        else:
            s = (num-1)*3 
            e = n      
        block_list = datesat_list.slice(s,e,1)
        collection = ee.ImageCollection.fromImages(block_list)
        NDVI_mosaic = collection.qualityMosaic('NDVI')
        mosaic_list = mosaic_list.add(NDVI_mosaic)

    ############################
        
    composite_datesat = ee.ImageCollection.fromImages(mosaic_list)
    
    return composite_datesat

composite_datesat_2021 = Potential_clearing_timeseries(datesat_QY_TGF_2021)

dif_NDVI_composite_datesat_2021 = count_VI_dif(composite_datesat_2021)
    
def PCT_sort(datesat_QY_TGF_2021,dif_NDVI_composite_datesat):
    
    median_doy = (dif_NDVI_composite_datesat.qualityMosaic('NDVI')).select('H_doy')

    def count_dif_doy(image):
        doy = median_doy
        dif_doy = (image.select('doy')).subtract(doy).abs()
        return image.addBands(dif_doy.rename('dif_doy'))
    
    #  datesat_QY_TGF_2021 增加 dif_doy 波段 （ doy NDVI dif_doy）
    datesat_QY_TGF_2021 = datesat_QY_TGF_2021.map(count_dif_doy)

    #按照 dif_doy 对二维数组进行 sort
    dif_doy = datesat_QY_TGF_2021.toArray().arraySlice(1,2,3)
    sorted_datesat_2021 =  datesat_QY_TGF_2021.toArray().arraySort(dif_doy)

    #取 dif_doy最小的6景 影像重构数据集,并且按照doy顺序排列
    filter_datesat = sorted_datesat_2021.arraySlice(0,0,6)
    doy = filter_datesat.arraySlice(1,0,1)
    filter_datesat_s = filter_datesat.arraySort(doy)
    
    return filter_datesat_s

#为了避免未识别的云和云阴影 ，通过3景提取 来获取 (optimal TimeSeries Construction)

filter_datesat_s_2021 = PCT_sort(datesat_QY_TGF_2021,dif_NDVI_composite_datesat_2021)
#filter_datesat_s_2022 = PCT_sort(datesat_QY_TGF_2022,dif_NDVI_composite_datesat_2022)

def get_PCT_datesat(filter_datesat_s):
    
    no = [k for k in range(0,6)]
    Name = ['NDVI'+str(k) for k in no]
    
    doy_collection = filter_datesat_s.arraySlice(1,0,1).arrayProject([0]).arrayFlatten([Name])
    NDVI_collection = filter_datesat_s.arraySlice(1,1,2).arrayProject([0]).arrayFlatten([Name])
    #t_collection = filter_datesat_s.arraySlice(1,1,2).arrayProject([0]).arrayFlatten([Name])
    zlist = ee.List([])

    for j in range(0,6):
        NDVI = NDVI_collection.select(Name[j]).rename('NDVI')
        doy = doy_collection.select(Name[j]).rename('doy')
      #  t = t_collection.select(Name[j]).rename('t')
       # img = NDVI.addBands(t).addBands(doy)
        img = doy.addBands(NDVI)
        zlist = zlist.add(img)

    PCT_datesat = ee.ImageCollection.fromImages(zlist)
    
    return PCT_datesat

15


In [41]:
# PCT_datesat_2021 格式为  ImageCollection   为潜在清除时段的 6景影像， 波段为（doy and NDVI）
PCT_datesat_2021 = get_PCT_datesat(filter_datesat_s_2021)


In [42]:
# dif_NDVI_composite_datesat 是 DIF_NDVI

DIF_NDVI_max = dif_NDVI_composite_datesat_2021.qualityMosaic('NDVI')
t_before = DIF_NDVI_max.select('doy_before') # t1
t_after =  DIF_NDVI_max.select('doy_after')  # t2

######## 拟合参数 ######
a0 = 0.34824
a1 = -0.06868
b1 = -0.22907
a2 = -0.01245
b2 = -0.0168

####y = a0 + a1*cos(2*pi*t/365) + b1*sin(2*pi*t/365) + a2*cos(4*pi*t/365) + b2*sin(4*pi*t/365)
y1 = ((t_before.multiply(2*math.pi/365)).cos().multiply(a1)).add((t_before.multiply(2*math.pi/365)).sin().multiply(b1))
y2 = ((t_before.multiply(4*math.pi/365)).cos().multiply(a2)).add((t_before.multiply(4*math.pi/365)).sin().multiply(b2))
y_before = (y1.add(y2).add(a0)).multiply(1000).toInt().divide(1000)

y3 = ((t_after.multiply(2*math.pi/365)).cos().multiply(a1)).add((t_after.multiply(2*math.pi/365)).sin().multiply(b1))
y4 = ((t_after.multiply(4*math.pi/365)).cos().multiply(a2)).add((t_after.multiply(4*math.pi/365)).sin().multiply(b2))
y_after = (y3.add(y4).add(a0)).multiply(1000).toInt().divide(1000)

Dy_thred = (y_before.subtract(y_after)).rename('NDVI')
#PCT_dif_NDVI = MM_2021.select('NDVI')
PCT_dif_NDVI = dif_NDVI_composite_datesat_2021.qualityMosaic('NDVI').select('NDVI')
dif_Dy_thred = PCT_dif_NDVI.subtract(Dy_thred)

###########get_last_img_NDVI############
n = datesat_QY_TGF_2021.size().getInfo()
datesat_list = datesat_QY_TGF_2021.toList(n)
img = ee.Image(datesat_list.get(n-1))
img_NDVI = img.select('NDVI')

#根据Dy_thred 和 img_NDVI 生成掩膜 提取未清除区域

#Non_clearing_mask_2021  = (img_NDVI.gt(0.25).And(dif_Dy_thred.lte(0.15))).Or(img_NDVI.gt(0.45))




In [43]:
# PCT_time_interval = DIF_NDVI_max.select('dif_d')
# Dy_thred = (PCT_time_interval.multiply(0.1).divide(60)).add(0.15)
# dif_Dy_thred1 = PCT_dif_NDVI.subtract(Dy_thred)

Non_clearing_mask_2021  = (img_NDVI.gt(0.32).And(dif_Dy_thred.lte(0.1))).Or(img_NDVI.gt(0.45))
#Non_clearing_mask_2021  = (img_NDVI.gt(0.25).And(dif_Dy_thred1.lt(0))).Or(img_NDVI.gt(0.45))
viz1 = {
    'bands':['doy_after'],
    'max':365,
    'min':150,
    'palette': ['276419','d7301f','ef6548','fc8d59','fdbb84','fdd49e','fee8c8','f7fcf0','e0f3db','ccebc5','a8ddb5','7bccc4','4eb3d3','2b8cbe','0868ac']
}
#生成 PCT_datesat 的 差值数据集
PCT_datesat_dif_2021 = count_VI_dif(PCT_datesat_2021)
MM_2021 = PCT_datesat_dif_2021.qualityMosaic('NDVI')

clearing_mask_2021 = Non_clearing_mask_2021.add(-1)
clearing_mask_2021 = clearing_mask_2021.where(clearing_mask_2021.lt(0),1)


clearing_region_2021 = MM_2021.updateMask(clearing_mask_2021)
Non_clearing_region_2021 = (MM_2021.updateMask(Non_clearing_mask_2021)).multiply(0)
merge_2021 = ee.ImageCollection([clearing_region_2021,Non_clearing_region_2021])

hhmc_shp = HHMC4_DZG
harvest_date_2021 = merge_2021.mosaic()


6


In [44]:

hhmc_shp = HHMC4_DZG
harvest_date_2021 = merge_2021.mosaic()
harvest_date_2021_clip = harvest_date_2021.clip(hhmc_shp)

QQ = geemap.Map()
QQ.addLayer(harvest_date_2021_clip,viz1,'harvest_date_2022_clip')

In [45]:
QQ

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [48]:
root_file = 'D:/hhmc_roi/hhmc2/roi_xz/'
out_dir = 'D:\hhmc_roi\SD_result\zasfc'
for i in range(1,10):
    roi_folder = root_file + 'HHK'+str(i)+'.shp'
    #print(roi_folder)
    roi = geemap.shp_to_ee(roi_folder)
    roi = roi.geometry()
    
    name = 'S2_HHKb_k' + str(i) + '.tif'
    filename = os.path.join(out_dir, name)
    print(filename)
    harvest_date_2021_k = harvest_date_2021.select('doy_after')
    geemap.ee_export_image(
        harvest_date_2021_k, filename=filename, scale=10, region=roi, file_per_band=False
    )


D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k1.tif
D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k2.tif
Generating URL ...
Please wait ...
Data downloaded to D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k2.tif
D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k3.tif
Generating URL ...
Please wait ...
Data downloaded to D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k3.tif
D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k4.tif
Generating URL ...
Please wait ...
Data downloaded to D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k4.tif
D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k5.tif
Generating URL ...
Please wait ...
Data downloaded to D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k5.tif
D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k6.tif
Generating URL ...
Please wait ...
Data downloaded to D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k6.tif
D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k7.tif
Generating URL ...
Please wait ...
Data downloaded to D:\hhmc_roi\SD_result\zasfc\S2_HHKb_k7.tif
D:\hhmc_roi\SD_resul